In [1]:
import argparse
# from dataclasses import dataclass
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
import os
import openai 
import json

API_SECRET_KEY = "sk-r0WeYOdkMjzYdnSxEcC8B931Aa904e4bBaCcAc2a57D803F1"
BASE_URL = "https://svip.xty.app/v1"
os.environ["OPENAI_API_KEY"] = API_SECRET_KEY
os.environ["OPENAI_API_BASE"] = BASE_URL
openai.api_key = os.environ['OPENAI_API_KEY']

CHROMA_PATH = "chroma_sentence_transformers"

PROMPT_TEMPLATE = """
背景信息：

{context}

---

请根据上文信息回答问题: {question}
"""

In [2]:

def load_txt(file_path):
    txt = open(file_path, 'r')
    lines=[]
    for line in txt:
        line = line.strip('\n').rstrip()
        lines.append(line)
    print("len lines",len(lines))
    return lines

def get_response(db,model,query_text:str):
    results = db.similarity_search_with_relevance_scores(query_text, k=3)
    print(results)
    if len(results) == 0 or results[0][1] < 0.7:
        print(f"Unable to find matching results.")
        return 

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    # print(prompt)

    # model = ChatOpenAI()
    response_text = model.predict(prompt)

    # sources = [doc.metadata.get("source", None) for doc, _score in results]
    # formatted_response = f"Response: {response_text}\nSources: {sources}"
    formatted_response=response_text
    
    # print(query_text)
    # print(formatted_response)
    
    return context_text,formatted_response

In [3]:
# embedding_function = OpenAIEmbeddings()
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)
model = ChatOpenAI(model_name='gpt-4', temperature=0.0) #choose gpt-4

/home/bjtuzky/.conda/envs/rag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_742523/3312384691.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)


In [ ]:

query_text = "/"
# query_list=load_txt('./query.txt')
# Prepare the DB.


data_QA=[]
context_text,res=get_response(db,model,query_text)
data={
    "instruction":query_text,
    "context":context_text,
    "output":res,
    "system":"你是一个法律方面的专家。"
}
print(data)
data_QA.append(data)
    
# with open('data_QA.json', 'w', encoding='utf-8') as f:
#     json.dump(data_QA, f, ensure_ascii=False, indent=4)


AttributeError: 'PersistentData' object has no attribute 'max_seq_id'